# week 03 part 3

## 1. setting up environment

In [109]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

import json

!pip install geopandas
!pip install geopy
!pip install beautifulsoup4
!pip install html5lib
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Library imported.")

Library imported.


## 2. Scrape thWikipedia page

In [110]:
 data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [111]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_table = pd.read_html(url,header=[0])
df = read_table[0]
df.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


## 3. modifying the dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [112]:
#rename columns
df = df.rename(index=str, columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'})
df.head()



,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is *Not assigned*

In [113]:
#drop not assigned
df = df[df.Borough !='Not assigned']
df.reset_index(drop=True,inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area, combine rows

In [114]:
#merge by postal code  
df = df.groupby('Postal code',as_index=False).agg(lambda x: ','.join(set(x.dropna())))

df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Borough not assigned neighborhood

In [115]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df['Borough']
#rename to make column name correct
df.rename(columns={"Postal code": "PostalCode"}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Show the number of rows

In [116]:
df.shape[0]

103

## 3.  create df containing geographical coordinates

In [117]:
# load the coordinates from the csv file on Coursera
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [118]:
#initiating l and l
df['Latitude'] = '0';
df['Longitude'] = '0';

#adding coordinates
for i in df.index:
    for j in df_geo.index:
        if df.iloc[i, 0] == df_geo.iloc[j, 0]:
            df.iloc[i, 3] = df_geo.iloc[j, 1]
            df.iloc[i, 4] = df_geo.iloc[j, 2]
            
df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


## 4. cluster the neighborhood whose Borough containing "Toronto"

In [119]:
dfa = df.copy()
dfa= dfa[df.Borough.str.contains("Toronto")]
dfa.tail()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
82,M6P,West Toronto,High Park / The Junction South,43.6616,-79.4648
83,M6R,West Toronto,Parkdale / Roncesvalles,43.649,-79.4563
84,M6S,West Toronto,Runnymede / Swansea,43.6516,-79.4844
85,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895
87,M7Y,East Toronto,Business reply mail Processing CentrE,43.6627,-79.3216


In [120]:
#set up and modify map
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = dfa['Latitude']
Y = dfa['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=5, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
dfa['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(dfa['Latitude'], dfa['Longitude'], dfa['Borough'], dfa['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map

## 5. find top 50 venues within a radius of 5000m w.r.t. Queen's Park / Ontario Provincial Government

In [121]:
lat = dfa.loc[85, 'Latitude'] # neighborhood latitude value, find Queen's location
long = dfa.loc[85, 'Longitude'] # neighborhood longitude value

neigh = dfa.loc[85, 'Neighborhood'] # neighborhood name

print('{} has a latitude and longitude value of {} and {}.'.format(neigh, 
                                                               lat, 
                                                              long))

Queen's Park / Ontario Provincial Government has a latitude and longitude value of 43.6623015 and -79.3894938.


In [122]:
#define foursquare credentials

CLIENT_ID = 'ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH' 
CLIENT_SECRET = 'W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)




Your credentails:
CLIENT_ID: ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH
CLIENT_SECRET:W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2


In [123]:
#define url of foursquare
LIMIT = 50
radius = 5000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH&client_secret=W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2&v=20180605&ll=43.6623015,-79.3894938&radius=5000&limit=50'

In [124]:
#get result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea97585211536324bacd30e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.70730154500004,
    'lng': -79.32740548666185},
   'sw': {'lat': 43.617301454999954, 'lng': -79.45158211333815}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a81ae339deb7d369fa7f146',
       'name': 'Hailed Coffee',
       'location': {'address': '44 Gerrard St W',
        'crossStreet': 'Yonge St',
        'lat': 43.65883296982352,
        'lng': -79.38368351986598,
        'labeledLatLngs': [{'label': 'display',
        

In [125]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [126]:
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize

nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.neighborhood','venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/dongningsun/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,neighborhood,categories,lat,lng
0,Hailed Coffee,NaN,Coffee Shop,43.658833,-79.383684
1,DanceLifeX Centre,NaN,Dance Studio,43.666956,-79.385297
2,Como En Casa,NaN,Mexican Restaurant,43.665160,-79.384796
3,Queen's Park,NaN,Park,43.663946,-79.392180
4,Neo Coffee Bar,NaN,Coffee Shop,43.660140,-79.385870


In [127]:
#provide how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 38 uniques categories.


## 6.Explore Neighborhood in Toronto

In [128]:
#define function to retrieve venues information for neighborhood in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [129]:
#run the function to provide top 50 venues around each neighborhood in Toronto within radius of 5000m
trt_vn = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])


Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [130]:
#provide the size of the returned venues
print(trt_vn.shape)
trt_vn.head()

(1693, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [131]:
#provide how many venues per each neighborhood
trt_vn.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,24,24,24,24,24,24


In [132]:
#print the amount of unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(trt_vn['Venue Category'].unique())))

There are 254 uniques categories.


## 7. Analyze Each Neighborhood

In [133]:
# one hot encoding
trt_onehot = pd.get_dummies(trt_vn[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
trt_onehot['Neighborhood'] = trt_vn['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [trt_onehot.columns[-1]] + list(trt_onehot.columns[:-1])
trt_onehot = trt_onehot[fixed_columns]

trt_grp = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_grp.head();
trt_grp.shape

(93, 254)

# pause here and go to the top 10 venues

In [134]:
#print the top 5 venues around each neighbothood with their recurrence frequency 
num_top_venues = 5

for hood in trt_grp['Neighborhood']:
    print("----"+hood+"----")
    temp = trt_grp[trt_grp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.33
1  Latin American Restaurant  0.33
2             Breakfast Spot  0.33
3                Yoga Studio  0.00
4          Martial Arts Dojo  0.00


----Alderwood / Long Branch----
          venue  freq
0   Pizza Place  0.25
1   Coffee Shop  0.12
2      Pharmacy  0.12
3           Gym  0.12
4  Skating Rink  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2  Convenience Store  0.05
3     Ice Cream Shop  0.05
4      Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0          Restaurant  0.08
1         Pizza Place  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.08

4            Steakhouse  0.05


----Kennedy Park / Ionview / East Birchmount Park----
                             venue  freq
0                 Department Store  0.33
1                      Coffee Shop  0.33
2                    Train Station  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Kensington Market / Chinatown / Grange Park----
                venue  freq
0                Café  0.10
1         Coffee Shop  0.08
2              Bakery  0.06
3  Mexican Restaurant  0.06
4         Gaming Cafe  0.04


----Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens----
                             venue  freq
0                      Pizza Place  0.25
1                             Park  0.25
2                   Sandwich Place  0.25
3                Mobile Phone Shop  0.25
4  Molecular Gastronomy Restaurant  0.00


----Lawrence Manor / Lawrence Heights----
                    venue  freq
0          Clothing Store  0.29
1      

                           venue  freq
0                 Sandwich Place  0.13
1                           Café  0.13
2                    Coffee Shop  0.09
3  Vegetarian / Vegan Restaurant  0.04
4                      BBQ Joint  0.04


----The Beaches----
                venue  freq
0   Health Food Store  0.25
1                 Pub  0.25
2               Trail  0.25
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----The Danforth West / Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----The Kingsway / Montgomery Road / Old Mill North----
           venue  freq
0          River  0.33
1           Park  0.33
2     Smoke Shop  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.10
1        Yoga Studio  0.05
2               Bank  0.05
3           B

In [135]:
#put information into dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [136]:
#get top 10 venues around each neighbothood with their recurrence frequency
# presented as a dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trt_grp['Neighborhood']

for ind in np.arange(trt_grp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trt_grp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Fabric Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,Alderwood / Long Branch,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Pub,Skating Rink,Gym,Colombian Restaurant,Drugstore,Donut Shop
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Pizza Place,Ice Cream Shop,Supermarket,Deli / Bodega,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Convenience Store
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Department Store,Dog Run
4,Bedford Park / Lawrence Manor East,Coffee Shop,Italian Restaurant,Pizza Place,Restaurant,Sandwich Place,Sushi Restaurant,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store


## 8. cluster neighborhood

In [70]:
# set number of clusters
kclusters = 5

trt_grp_clustering = trt_grp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trt_grp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [67]:
trt_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:


trt_merged = dfa

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
trt_merged = trt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
trt_merged.drop('Cluster',axis = 1)

trt_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6764,-79.293,0,1,Health Food Store,Pub,Trail,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
41,M4K,East Toronto,The Danforth West / Riverdale,43.6796,-79.3522,0,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,M4L,East Toronto,India Bazaar / The Beaches West,43.669,-79.3156,0,1,Park,Fast Food Restaurant,Pub,Liquor Store,Brewery,Sandwich Place,Italian Restaurant,Burrito Place,Ice Cream Shop,Restaurant
43,M4M,East Toronto,Studio District,43.6595,-79.3409,0,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,1,3,Park,Swim School,Bus Line,Fabric Shop,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [103]:

trt_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6764,-79.293,1,Health Food Store,Pub,Trail,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
41,M4K,East Toronto,The Danforth West / Riverdale,43.6796,-79.3522,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,M4L,East Toronto,India Bazaar / The Beaches West,43.669,-79.3156,1,Park,Fast Food Restaurant,Pub,Liquor Store,Brewery,Sandwich Place,Italian Restaurant,Burrito Place,Ice Cream Shop,Restaurant
43,M4M,East Toronto,Studio District,43.6595,-79.3409,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,3,Park,Swim School,Bus Line,Fabric Shop,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [137]:

# create map
map_clusters = folium.Map(location=[43.65, -79.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
centroid_colors = ['red','green''yellow','blue','black']

rainbow = [next(iter(centroid_colors),i) for i in range(kclusters)]
#it is very interesting that colors.rgb2hex is not foundable.


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trt_merged['Latitude'], trt_merged['Longitude'], trt_merged['Neighborhood'], trt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster Labels' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 9. Examine clusters

In [138]:
trt_merged.loc[trt_merged['Cluster Labels'] == 0, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [139]:
trt_merged.loc[trt_merged['Cluster Labels'] == 1, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Health Food Store,Pub,Trail,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
41,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,East Toronto,1,Park,Fast Food Restaurant,Pub,Liquor Store,Brewery,Sandwich Place,Italian Restaurant,Burrito Place,Ice Cream Shop,Restaurant
43,East Toronto,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
45,Central Toronto,1,Park,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Convenience Store,Gym,Doner Restaurant,Dog Run
46,Central Toronto,1,Clothing Store,Coffee Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park,Sporting Goods Shop
47,Central Toronto,1,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Gym,Café,Thai Restaurant,Toy / Game Store,Italian Restaurant,Coffee Shop
49,Central Toronto,1,Pub,Coffee Shop,Pizza Place,American Restaurant,Sushi Restaurant,Bank,Sports Bar,Fried Chicken Joint,Restaurant,Supermarket
51,Downtown Toronto,1,Coffee Shop,Café,Pub,Bakery,Italian Restaurant,Pharmacy,Pizza Place,Restaurant,Beer Store,Jewelry Store
52,Downtown Toronto,1,Coffee Shop,Restaurant,Sushi Restaurant,Men's Store,Japanese Restaurant,Yoga Studio,Gastropub,Caribbean Restaurant,Poke Place,Pizza Place


In [140]:
trt_merged.loc[trt_merged['Cluster Labels'] == 2, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Women's Store,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [141]:
trt_merged.loc[trt_merged['Cluster Labels'] == 3, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Park,Swim School,Bus Line,Fabric Shop,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
48,Central Toronto,3,Park,Playground,Tennis Court,Restaurant,Donut Shop,Drugstore,Doner Restaurant,Dog Run,Distribution Center,Curling Ice
50,Downtown Toronto,3,Park,Playground,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [142]:
trt_merged.loc[trt_merged['Cluster Labels'] == 4, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
